In [2]:
!pip install -qq s3fs

ERROR: boto3 1.12.34 has requirement botocore<1.16.0,>=1.15.34, but you'll have botocore 1.19.52 which is incompatible.
ERROR: awscli 1.18.34 has requirement botocore==1.15.34, but you'll have botocore 1.19.52 which is incompatible.


In [1]:
import os
import glob
#import s3fs

In [3]:
#data_dir= './aclImdb_v1/aclImdb'
data_dir= 's3://sagemaker-ap-south-1-831906679170/aclImdb/'
folders= ['train', 'test']
fs = s3fs.S3FileSystem()

def read_data(path):
    #srch_path= os.path.join(path, "*.txt")
    #print("Searching for "+ srch_path)
    #files= glob.glob(srch_path)
    files= fs.ls(path)
    print("number of files: "+ str(len(files)))
    data= []
    for file in files:
        with fs.open(file, 'r') as fp:
            content= fp.read()
            data.append(content)
    
    return data

In [4]:
d='s3://sagemaker-ap-south-1-831906679170/aclImdb/train/pos/'
fs.ls(d)[:5]

['sagemaker-ap-south-1-831906679170/aclImdb/train/pos/0_9.txt',
 'sagemaker-ap-south-1-831906679170/aclImdb/train/pos/10000_8.txt',
 'sagemaker-ap-south-1-831906679170/aclImdb/train/pos/10001_10.txt',
 'sagemaker-ap-south-1-831906679170/aclImdb/train/pos/10002_7.txt',
 'sagemaker-ap-south-1-831906679170/aclImdb/train/pos/10003_8.txt']

In [5]:
train_pos_reviews= read_data(os.path.join(data_dir,'train','pos'))
train_neg_reviews= read_data(os.path.join(data_dir,'train','neg'))

number of files: 12500
number of files: 12500


In [4]:
from sklearn.utils import shuffle

In [7]:
train_reviews= train_pos_reviews + train_neg_reviews
train_labels= [1]* len(train_pos_reviews) + [0] * len(train_neg_reviews)

train_reviews, train_labels= shuffle(train_reviews, train_labels, random_state= 711)

In [13]:
import pickle

with open("./train_reviews.pkl",'wb') as fp:
    pickle.dump(train_reviews, fp)
    
with open("./train_labels.pkl", "wb") as fp:
    pickle.dump(train_labels, fp)

In [8]:
train_labels[:10]

[1, 1, 1, 1, 0, 1, 1, 0, 0, 1]

In [2]:
!pip install -qq transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 1.0.61 requires nvidia-ml-py3, which is not installed.


In [3]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np

In [5]:
MODEL_NAME = 'bert-base-uncased'
tokenizer= BertTokenizer.from_pretrained(MODEL_NAME)

In [8]:
sentences= ["I like it :) ", "Go baby GO BABY it is a great movie"]
for sent in sentences:
    tokens= tokenizer.tokenize(sent)
    print(tokens)

['i', 'like', 'it', ':', ')']
['go', 'baby', 'go', 'baby', 'it', 'is', 'a', 'great', 'movie']


In [9]:
tokenizer(sentences,
         padding= 'max_length',
         max_length= 15,
         return_tensors= 'pt',
         return_token_type_ids= False
         )

{'input_ids': tensor([[ 101, 1045, 2066, 2009, 1024, 1007,  102,    0,    0,    0,    0,    0,
            0,    0,    0],
        [ 101, 2175, 3336, 2175, 3336, 2009, 2003, 1037, 2307, 3185,  102,    0,
            0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [7]:
print(tokenizer.pad_token_id)
print(tokenizer.sep_token_id)
print(tokenizer.cls_token_id)

0
102
101


In [8]:
from torch.utils.data import Dataset, DataLoader
from bs4 import BeautifulSoup

In [9]:
class ReviewDataset(Dataset):
    
    def __init__(self, reviews, labels, tokenizer, max_len):
        self.reviews= reviews
        self.labels= labels
        self.tokenizer= tokenizer
        self.max_len= max_len
        
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review= self.reviews[item]
        review= BeautifulSoup(review, "html.parser").get_text()
        label= self.labels[item]
        
        encodings= tokenizer(
                review,
                padding= 'max_length',
                max_length= self.max_len,
                truncation= True,
                return_tensors= 'pt'
                )
        
        return {
            #'review': review,
            'encoding': encodings,
            'label': torch.tensor(label, dtype=torch.long)
        }

In [13]:
chk_data= ReviewDataset(sentences, [1,0], tokenizer, 5)

In [14]:
chk_data[1]

{'encoding': {'input_ids': tensor([[ 101, 2175, 3336, 2175,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])},
 'label': tensor(0)}

In [10]:
def createDataLoader(dataset, tokenizer, max_len= 150, batch_size= 32, num_workers=0):
    '''
    dataset is a dictionary with 2 keys 'data', 'labels'
    '''
    encoded_dataset= ReviewDataset(dataset['data'], dataset['labels'], tokenizer, max_len)
    
    return DataLoader(encoded_dataset, batch_size= batch_size, num_workers= num_workers)

In [1]:
import pickle

with open("./data/train_reviews.pkl",'rb') as fp:
    labeled_reviews= pickle.load(fp)

with open("./data/train_labels.pkl",'rb') as fp:
    labeled_labels= pickle.load(fp)

print(type(labeled_reviews))
print(len(labeled_reviews))

print(type(labeled_labels))
print(len(labeled_labels))

<class 'list'>
25000
<class 'list'>
25000


In [2]:
from sklearn.model_selection import train_test_split

train_reviews, val_reviews, train_labels, val_labels= train_test_split(
    labeled_reviews,
    labeled_labels,
    random_state= 711,
    train_size= 0.8
    )

In [3]:
print(len(train_reviews))
print(train_reviews[0])
print()
print(len(val_reviews))
print(val_reviews[0])

20000
Like last year, I didn't manage to sit through the whole thing. Okay, so Chris Rock as a host was a good choice because he was vaguely engaging. Or rather, out of all the total bores packed into the theatre, he at least wasn't in the Top 10 Most Boring. A lot of the presenters, on the other hand, were in this coveted Top 10. I hadn't known that the whole thing had been done by autocue (although I knew it was scripted) but it was really terrible to see these supposedly good actors unable to insert expression, look away from the cue and stumble over simple words (Natalie Portmanif there's no director, she's gone). The Night of Fancy Dresses and Boring Speeches was long and tedious, Beyonce Knowles butchered some good songs and there were very few decent acceptance speeches and clips. Adam Sandler wins the Worst Presenter award.<br /><br />For helping me write this review I'd like to thank my Mum, my Dad, my lawyers and my pedicurist for all believing in me, and I'd like to point o

In [5]:
with open("./data/train_reviews_v2.pkl",'wb') as fp:
    pickle.dump(train_reviews, fp)
    
with open("./data/val_reviews_v2.pkl", "wb") as fp:
    pickle.dump(val_reviews, fp)
    
with open("./data/train_labels_v2.pkl",'wb') as fp:
    pickle.dump(train_labels, fp)
    
with open("./data/val_labels_v2.pkl", "wb") as fp:
    pickle.dump(val_labels, fp)

In [14]:
train_dataset= {'data': train_reviews, 'labels':train_labels}
train_dataloader= createDataLoader(train_dataset, tokenizer)

In [15]:
dl_item= next(iter(train_dataloader))
print(len(dl_item))

2


In [16]:
dl_item

{'encoding': {'input_ids': tensor([[[  101,  2066,  2197,  ...,  1012,  1996,   102]],
  
          [[  101,  2023,  2143,  ...,  3302,  1010,   102]],
  
          [[  101,  1037,  6703,  ...,  2057,  2963,   102]],
  
          ...,
  
          [[  101,  2074,  2128,  ...,  1011,  6052,   102]],
  
          [[  101,  8870, 23185,  ...,  2860,  2860,   102]],
  
          [[  101,  8235,  2338,  ...,     0,     0,     0]]]),
  'token_type_ids': tensor([[[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          ...,
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]],
  
          [[0, 0, 0,  ..., 0, 0, 0]]]),
  'attention_mask': tensor([[[1, 1, 1,  ..., 1, 1, 1]],
  
          [[1, 1, 1,  ..., 1, 1, 1]],
  
          [[1, 1, 1,  ..., 1, 1, 1]],
  
          ...,
  
          [[1, 1, 1,  ..., 1, 1, 1]],
  
          [[1, 1, 1,  ..., 1, 1, 1]],
  
          [[1, 1, 1,  ..., 0, 0, 0]]])},
 'lab

In [17]:
ip_tensor= dl_item['encoding']['input_ids']
ip_tensor.shape

torch.Size([32, 1, 150])

In [18]:
dl_item['label'].shape

torch.Size([32])

In [6]:
bert_model = BertModel.from_pretrained(MODEL_NAME)

In [19]:
ip_id_tensor= dl_item['encoding']['input_ids']
attention_mask_tensor= dl_item['encoding']['attention_mask']
print(ip_id_tensor.shape)
print(attention_mask_tensor.shape)

torch.Size([32, 1, 150])
torch.Size([32, 1, 150])


In [20]:
ip_id_tensor= ip_id_tensor.squeeze()
attention_mask_tensor= attention_mask_tensor.squeeze()
print(ip_id_tensor.shape)
print(attention_mask_tensor.shape)

torch.Size([32, 150])
torch.Size([32, 150])


In [30]:
ret_val= bert_model(
  input_ids= ip_id_tensor, 
  attention_mask= attention_mask_tensor
)

In [31]:
type(ret_val)


transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions

In [32]:
ret_val.last_hidden_state.shape

torch.Size([32, 150, 768])

In [33]:
ret_val.pooler_output.shape

torch.Size([32, 768])

In [34]:
ret_val.keys()

odict_keys(['last_hidden_state', 'pooler_output'])